In [1]:
## import relevant packages
import pandas as pd
import urllib.request as urllib
import requests
from bs4 import BeautifulSoup

In [2]:
## webscraping cell

## make dataframes to be filled
season = []
week = []
team1 = []
team2 = []
manager1 = []
manager2 = []
team1score = []
team2score = []
team1record = []
team2record = []
team1streak = []
team2streak = []
team1place = []
team2place = []
team1win = []
team2win = []

## url setup
site1 = 'https://fantasy.nfl.com/league/2461806/history/'
site2 = '/schedule?gameSeason='
site3 = '&leagueId=2461806&scheduleDetail='
site4 = '&scheduleType=week&standingsTab=schedule'
## loop from 2014 to 2020, weeks 1 to 16
for i in range(2020,2021):
    for y in range(1,17):
        year = str(i)
        week_num = str(y)
        
        ## create soup object for every week of every season
        site = site1 + year + site2 + year + site3 + week_num + site4 
        html = urllib.urlopen(site)
        soup = BeautifulSoup(html.read())
        
        ## isolate html containing matchup data
        li_tags = soup.find_all("li",{"class": "matchup"})
        for x in li_tags:
            teamNames = x.find_all("a",{"class": "teamName"})
            names = x.find_all("li",{"class": "name"})
            scores = x.find_all("div",{"class": "teamTotal"})
            records = x.find_all("span",{"class": "teamRecord"})
            streak = x.find_all("li",{"class": "streak"})
            places = x.find_all("span",{"class": "teamRank"})
            
            season.append(i)
            week.append(y)
        
            team1.append(teamNames[0].get_text())
            team2.append(teamNames[1].get_text())
        
            manager1.append(names[0].get_text())
            manager2.append(names[1].get_text())
       
            team1score.append(scores[0].get_text())
            team2score.append(scores[1].get_text())
        
            team1record.append(records[0].get_text())
            team2record.append(records[1].get_text())
        
            team1streak.append(streak[0].get_text().split()[2])
            team2streak.append(streak[1].get_text().split()[2])
       
            team1place.append(places[0].get_text().replace("(","").replace(")",""))
            team2place.append(places[1].get_text().replace("(","").replace(")",""))
            
            ## if team 1 scored more than team 2, then team 1 won the game
            if float(scores[0].get_text()) > float(scores[1].get_text()):
                team1win.append(1);
                team2win.append(0);
            else:
                team1win.append(0);
                team2win.append(1);


In [15]:
## create main dataframe
df = pd.DataFrame()
df["Season"] = season
df["Week"] = week
df["Team_1"] = team1
df["Team_2"] = team2
df["Manager_1"] = manager1
df["Manager_2"] = manager2
df["Team_1_Score"] = team1score
df["Team_1_Score"] = df["Team_1_Score"].astype(float)
df["Team_2_Score"] = team2score
df["Team_2_Score"] = df["Team_2_Score"].astype(float)
df["Margin"] = abs(df["Team_1_Score"] - df["Team_2_Score"])
df["Sum"] = df["Team_1_Score"] + df["Team_2_Score"]
df["Team_1_Streak"] = team1streak
df["Team_2_Streak"] = team2streak
df["Team_1_Record"] = team1record
df["Team_2_Record"] = team2record
df["Team_1_Place"] = team1place
df["Team_1_Place"] = df["Team_1_Place"].astype(float)
df["Team_2_Place"] = team2place
df["Team_2_Place"] = df["Team_2_Place"].astype(float)
df["Team_1_Win"] = team1win
df["Team_2_Win"] = team2win
df["Season_Week"] = df["Season"].astype(str) + " " + df["Week"].astype(str)


In [22]:
## create inverse dataframe, swapping team 1 and team 2 data
swap = pd.DataFrame()
swap["Season"] = season
swap["Week"] = week
swap["Team_1"] = team2
swap["Team_2"] = team1
swap["Manager_1"] = manager2
swap["Manager_2"] = manager1
swap["Team_1_Score"] = team2score
swap["Team_1_Score"] = swap["Team_1_Score"].astype(float)
swap["Team_2_Score"] = team1score
swap["Team_2_Score"] = swap["Team_2_Score"].astype(float)
swap["Team_1_Streak"] = team2streak
swap["Team_2_Streak"] = team1streak
swap["Team_1_Record"] = team2record
swap["Team_2_Record"] = team1record
swap["Team_1_Place"] = team2place
swap["Team_1_Place"] = swap["Team_1_Place"].astype(float)
swap["Team_2_Place"] = team1place
swap["Team_2_Place"] = swap["Team_2_Place"].astype(float)
swap["Team_1_Win"] = team2win
swap["Team_2_Win"] = team1win
swap["Season_Week"] = swap["Season"].astype(str) + " " + swap["Week"].astype(str)
swap["Margin"] = abs(swap["Team_1_Score"] - swap["Team_2_Score"])
swap["Sum"] = swap["Team_1_Score"] + swap["Team_2_Score"]

In [23]:
## double has all games twice, once with team 1 as team 1 and once with team 2 as team 1
double = df.append(swap)
## df of regular season matchups
regular_season = double[(double["Week"] != 16)  & (double["Week"] != 15) & (double["Week"] != 14)]

In [24]:
## number of games over 100 points
double[double["Team_1_Score"] > 120].groupby("Manager_1").count()["Season"]

Manager_1
Alex, Davis      6
Ayush            4
Chris, Kanish    4
Ishaan           2
Joe              5
Krishanu         4
Mikey            5
Nandha           7
Scott            2
Steve            4
Zach             6
Name: Season, dtype: int64

In [31]:
## number of games won by single digits
double[(double["Margin"] < 10) & (double["Team_1_Win"] == 1)].groupby("Manager_1").count()["Season"].sort_values()

Manager_1
Ankith           1
Joe              1
Scott            1
Alex, Davis      2
Mikey            2
Steve            2
Zach             2
Chris, Kanish    3
Krishanu         4
Nandha           5
Name: Season, dtype: int64

In [30]:
## number of games won as an underdog
double[(double["Team_1_Place"] > double["Team_2_Place"]) & double["Team_1_Win"] == 1].groupby("Manager_1").count()["Season"].sort_values()

Manager_1
Alex, Davis    1
Zach           1
Ishaan         2
Krishanu       2
Nandha         2
Ankith         3
Joe            3
Mikey          3
Steve          3
Scott          4
Name: Season, dtype: int64

In [32]:
## longest winning streaks (exlcuding double digit streaks)
regular_season[regular_season["Team_1_Win"] == 1].sort_values("Team_1_Streak", ascending=False).head(20)

,Season,Week,Team_1,Team_2,Manager_1,Manager_2,Team_1_Score,Team_2_Score,Margin,Sum,Team_1_Streak,Team_2_Streak,Team_1_Record,Team_2_Record,Team_1_Place,Team_2_Place,Team_1_Win,Team_2_Win,Season_Week
30,2020,6,Belichick Yourself,Dreams are For Idiots,"Chris, Kanish",Scott,138.54,95.28,43.26,233.82,W5,L1,5-1-0,1-5-0,1.0,12.0,1,0,2020 6
58,2020,10,Showtyme,The Wang Gang,Nandha,Steve,118.76,113.60,5.16,232.36,W5,L1,8-2-0,6-4-0,1.0,2.0,1,0,2020 10
49,2020,9,Showtyme,4-0 Dream,Nandha,Ankith,127.50,76.30,51.20,203.80,W4,L5,7-2-0,2-7-0,1.0,11.0,1,0,2020 9
28,2020,5,Belichick Yourself,Bok Choy Babes,"Chris, Kanish",Zach,96.98,91.72,5.26,188.70,W4,L1,4-1-0,2-3-0,2.0,7.0,1,0,2020 5
65,2020,11,Dreams are For Idiots,4-0 Dream,Scott,Ankith,93.04,66.58,26.46,159.62,W3,L1,4-7-0,3-8-0,10.0,11.0,1,0,2020 11
37,2020,7,Number Juan,Dreams are For Idiots,Mikey,Scott,105.92,60.22,45.70,166.14,W3,L2,4-3-0,1-6-0,8.0,12.0,1,0,2020 7
53,2020,9,The Wang Gang,Bok Choy Babes,Steve,Zach,115.82,89.24,26.58,205.06,W3,L1,6-3-0,5-4-0,2.0,5.0,1,0,2020 9
68,2020,12,her,4-0 Dream,Krishanu,Ankith,96.80,68.64,28.16,165.44,W3,L2,8-4-0,3-9-0,2.0,11.0,1,0,2020 12
46,2020,8,Showtyme,Ishaan,Nandha,Ishaan,81.36,72.04,9.32,153.40,W3,L4,6-2-0,2-6-0,1.0,11.0,1,0,2020 8
18,2020,4,Belichick Yourself,Aint Talkin' Bout Chubb,"Chris, Kanish","Alex, Davis",100.20,96.88,3.32,197.08,W3,L2,3-1-0,2-2-0,4.0,5.0,1,0,2020 4


In [37]:
## least points allowed on season
regular_season.groupby("Manager_1").sum()["Team_2_Score"].sort_values()

Manager_1
Alex, Davis      1199.68
Krishanu         1309.10
Nandha           1323.48
Ayush            1331.84
Scott            1337.76
Steve            1345.46
Zach             1352.32
Ankith           1352.76
Mikey            1354.80
Ishaan           1361.54
Chris, Kanish    1412.44
Joe              1555.82
Name: Team_2_Score, dtype: float64

In [38]:
## most points scored in a week
double.sort_values("Team_1_Score", ascending=False).head(20)

,Season,Week,Team_1,Team_2,Manager_1,Manager_2,Team_1_Score,Team_2_Score,Margin,Sum,Team_1_Streak,Team_2_Streak,Team_1_Record,Team_2_Record,Team_1_Place,Team_2_Place,Team_1_Win,Team_2_Win,Season_Week
91,2020,16,her,Belichick Yourself,Krishanu,"Chris, Kanish",169.28,71.44,97.84,240.72,L1,L2,8-5-0,7-6-0,4.0,6.0,1,0,2020 16
41,2020,7,The Wang Gang,C9H13NO3,Steve,Ayush,163.44,81.60,81.84,245.04,W1,L2,4-3-0,4-3-0,5.0,6.0,1,0,2020 7
10,2020,2,C9H13NO3,4-0 Dream,Ayush,Ankith,155.28,98.20,57.08,253.48,W2,L2,2-0-0,0-2-0,1.0,11.0,1,0,2020 2
6,2020,2,her,The Wang Gang,Krishanu,Steve,153.22,122.44,30.78,275.66,W2,L1,2-0-0,1-1-0,3.0,5.0,1,0,2020 2
85,2020,15,Aint Talkin' Bout Chubb,C9H13NO3,"Alex, Davis",Ayush,153.02,98.16,54.86,251.18,W2,W2,8-5-0,8-5-0,3.0,2.0,1,0,2020 15
0,2020,1,Aint Talkin' Bout Chubb,Bok Choy Babes,"Alex, Davis",Zach,152.06,113.50,38.56,265.56,W1,L1,1-0-0,0-1-0,1.0,7.0,1,0,2020 1
40,2020,7,Showtyme,Belichick Yourself,Nandha,"Chris, Kanish",151.84,106.02,45.82,257.86,W2,L1,5-2-0,5-2-0,1.0,4.0,1,0,2020 7
21,2020,4,Bok Choy Babes,Number Juan,Zach,Mikey,150.14,100.14,50.00,250.28,W1,L1,2-2-0,1-3-0,7.0,10.0,1,0,2020 4
25,2020,5,C9H13NO3,Hernandez Hit Men,Ayush,Joe,149.50,108.72,40.78,258.22,W1,L4,4-1-0,1-4-0,1.0,11.0,1,0,2020 5
15,2020,3,The Wang Gang,Hernandez Hit Men,Steve,Joe,145.10,93.20,51.90,238.30,W1,L2,2-1-0,1-2-0,3.0,10.0,1,0,2020 3


In [40]:
## largest margin of victory
double[double["Team_1_Win"] == 1].sort_values("Margin", ascending=False).head(20)

,Season,Week,Team_1,Team_2,Manager_1,Manager_2,Team_1_Score,Team_2_Score,Margin,Sum,Team_1_Streak,Team_2_Streak,Team_1_Record,Team_2_Record,Team_1_Place,Team_2_Place,Team_1_Win,Team_2_Win,Season_Week
91,2020,16,her,Belichick Yourself,Krishanu,"Chris, Kanish",169.28,71.44,97.84,240.72,L1,L2,8-5-0,7-6-0,4.0,6.0,1,0,2020 16
57,2020,10,4-0 Dream,Ishaan,Ankith,Ishaan,117.78,22.88,94.90,140.66,W1,L6,3-7-0,2-8-0,10.0,12.0,1,0,2020 10
41,2020,7,The Wang Gang,C9H13NO3,Steve,Ayush,163.44,81.60,81.84,245.04,W1,L2,4-3-0,4-3-0,5.0,6.0,1,0,2020 7
55,2020,10,Aint Talkin' Bout Chubb,Hernandez Hit Men,"Alex, Davis",Joe,138.38,61.02,77.36,199.40,W1,L1,6-4-0,3-7-0,4.0,9.0,1,0,2020 10
35,2020,6,Number Juan,Ishaan,Mikey,Ishaan,127.00,58.32,68.68,185.32,W2,L2,3-3-0,2-4-0,8.0,9.0,1,0,2020 6
31,2020,6,Bok Choy Babes,Hernandez Hit Men,Zach,Joe,121.08,57.72,63.36,178.80,W1,L5,3-3-0,1-5-0,6.0,11.0,1,0,2020 6
83,2020,15,Hernandez Hit Men,Bok Choy Babes,Joe,Zach,132.72,75.46,57.26,208.18,W3,L2,6-7-0,6-7-0,8.0,9.0,1,0,2020 15
10,2020,2,C9H13NO3,4-0 Dream,Ayush,Ankith,155.28,98.20,57.08,253.48,W2,L2,2-0-0,0-2-0,1.0,11.0,1,0,2020 2
85,2020,15,Aint Talkin' Bout Chubb,C9H13NO3,"Alex, Davis",Ayush,153.02,98.16,54.86,251.18,W2,W2,8-5-0,8-5-0,3.0,2.0,1,0,2020 15
11,2020,2,Aint Talkin' Bout Chubb,Dreams are For Idiots,"Alex, Davis",Scott,118.50,64.56,53.94,183.06,W2,L2,2-0-0,0-2-0,2.0,12.0,1,0,2020 2


In [42]:
## most games won by 30+ points
double[(double["Team_1_Win"] == 1) & (double["Margin"] > 30)].groupby("Manager_1").count()["Season"].sort_values()

Manager_1
Ishaan           1
Ankith           2
Chris, Kanish    2
Mikey            2
Scott            2
Steve            2
Ayush            3
Joe              3
Krishanu         3
Nandha           3
Zach             5
Alex, Davis      7
Name: Season, dtype: int64

In [44]:
## longest losing streaks (exlcuding double digit streaks)
regular_season[regular_season["Team_1_Win"] == 0].sort_values("Team_1_Streak", ascending=False).head(20)

,Season,Week,Team_1,Team_2,Manager_1,Manager_2,Team_1_Score,Team_2_Score,Margin,Sum,Team_1_Streak,Team_2_Streak,Team_1_Record,Team_2_Record,Team_1_Place,Team_2_Place,Team_1_Win,Team_2_Win,Season_Week
57,2020,10,Ishaan,4-0 Dream,Ishaan,Ankith,22.88,117.78,94.90,140.66,L6,W1,2-8-0,3-7-0,12.0,10.0,0,1,2020 10
38,2020,7,Hernandez Hit Men,her,Joe,Krishanu,103.62,105.40,1.78,209.02,L6,W2,1-6-0,5-2-0,11.0,3.0,0,1,2020 7
31,2020,6,Hernandez Hit Men,Bok Choy Babes,Joe,Zach,57.72,121.08,63.36,178.80,L5,W1,1-5-0,3-3-0,11.0,6.0,0,1,2020 6
49,2020,9,4-0 Dream,Showtyme,Ankith,Nandha,76.30,127.50,51.20,203.80,L5,W4,2-7-0,7-2-0,11.0,1.0,0,1,2020 9
48,2020,9,Ishaan,Dreams are For Idiots,Ishaan,Scott,110.30,111.80,1.50,222.10,L5,W1,2-7-0,2-7-0,10.0,12.0,0,1,2020 9
20,2020,4,Dreams are For Idiots,her,Scott,Krishanu,96.62,120.22,23.60,216.84,L4,W1,0-4-0,3-1-0,12.0,3.0,0,1,2020 4
46,2020,8,Ishaan,Showtyme,Ishaan,Nandha,72.04,81.36,9.32,153.40,L4,W3,2-6-0,6-2-0,11.0,1.0,0,1,2020 8
25,2020,5,Hernandez Hit Men,C9H13NO3,Joe,Ayush,108.72,149.50,40.78,258.22,L4,W1,1-4-0,4-1-0,11.0,1.0,0,1,2020 5
42,2020,8,4-0 Dream,Belichick Yourself,Ankith,"Chris, Kanish",78.40,131.84,53.44,210.24,L4,W1,2-6-0,6-2-0,10.0,2.0,0,1,2020 8
75,2020,13,4-0 Dream,C9H13NO3,Ankith,Ayush,87.60,107.72,20.12,195.32,L3,W2,3-10-0,8-5-0,11.0,2.0,0,1,2020 13
